# Tinkering with Keras

The goal of this notebook is to store useful insights made through my learning process of [Keras](https://keras.io/). Since I will be closely working with a colleague who uses Theano, the idea is to try to implement all the functionalities to work seamlessly with both backends. 

Below are shown some of the functionalities that I found imperceptible from the official documentation.

### Evaluate variables.

In [1]:
%clear -f
import keras.backend as K

x = K.variable(42.)

# Solution 1:
sess = K.get_session()
print sess.run(x)

# Solution 2:
print K.eval(x)

Using TensorFlow backend.


42.0
42.0


### Feed placeholders and evaluate functions.

In [2]:
%clear -f
import keras.backend as K
import numpy as np

x = K.placeholder(ndim=1)
y = 2 * x
feed = np.array([2])

# Solution 1:
sess = K.get_session()
print sess.run(y, { x : feed})

# Solution 2 (general for tensorflow and theano):
f = K.function([x], [y])
print f([feed])[0]

[ 4.]
[ 4.]


### Compute custom gradients.

In [3]:
%clear -f
import numpy as np
import keras.backend as K
from keras.layers import Input, Dense, Activation
from keras.models import Model
from keras import optimizers, losses

# Define the model and generate the data.
print "Our model is y = 3 * x1 + 2 * x2"
num_data = 1000
x_train = np.random.rand(num_data, 2)
y_train = x_train * np.matrix([[3], [2]])
print "Data generated."
x = Input(shape=(2,), name='x')
y = Dense(1, activation='linear', use_bias=False)(x)
model = Model(inputs=x, outputs=y)

opt = optimizers.Adam(0.1)
loss = losses.mean_squared_error

model.compile(opt, loss)

# Retreive the handle to trainable weights, which should be equal to the dy_dx.
w = model.trainable_weights[0]

dy_dx = K.gradients(y, x) 
f = K.function(inputs=[x], outputs=dy_dx)
fx = f([np.ones([1, 2])])[0]  # Input is irrelevant, since we are computing the gradient of a linear function.
print "Before training:"
print "dy_dx = ", fx, ", w = ", K.eval(w).T 
model.fit(x_train, y_train, epochs=500, batch_size=num_data, verbose=0)
print "\nAfter training:"
print "dy_dx = ", f([np.ones([1, 2])])[0], ", w = ", K.eval(w).T 

# Calculate the gradient manually.
y_true = K.placeholder(shape=(None, 1))
dJ_dw = K.gradients(loss(y, y_true), w) 
f = K.function(inputs=[x, y_true], outputs=dJ_dw)

print "\ndJ_dw (should be around 0):"
print f([x_train, y_train])[0].T

Our model is y = 3 * x1 + 2 * x2
Data generated.
Before training:
dy_dx =  [[ 1.38454044 -0.6651383 ]] , w =  [[ 1.38454044 -0.6651383 ]]

After training:
dy_dx =  [[ 2.99999952  2.00000024]] , w =  [[ 2.99999952  2.00000024]]

dJ_dw (should be around 0):
[[ -1.80480551e-04  -8.61882290e-05]]


### Pass a custom gradient to the optimizer.

In [4]:
%clear -f
import tensorflow as tf
import numpy as np
import keras.backend as K
from keras.layers import Input, Dense, Activation
from keras.models import Model
from keras import optimizers, losses

# Define the model and generate the data.
print "Our model is y = 3 * x1 + 2 * x2 + 5"
num_data = 1000
x_train = np.random.rand(num_data, 2)
y_train = x_train * np.matrix([[3], [2]]) + 5
print "Data generated."
x = Input(shape=(2,), name='x')
y = Dense(1, activation='linear')(x)
model = Model(inputs=x, outputs=y)

# Retreive the handle to trainable weights. In this case it is a list of variables [W, b].
w = model.trainable_weights
store_w = {l: l.get_weights() for l in model.layers}

print "Initial weights:", [K.eval(i).T for i in w]

# opt = optimizers.Adam(0.1)
opt = tf.train.AdamOptimizer(0.1)
# Output placeholder is needed for to define the loss.
y_true = K.placeholder(shape=(None, 1))
loss = losses.mean_squared_error(y, y_true)

grads_and_vars = zip(tf.gradients(loss, w), w)
op = opt.apply_gradients(grads_and_vars)

print "\nFirst approach:"
f = K.function(inputs=[x, y_true], outputs=[op])
for i in range(1000):
    f([x_train, y_train])
    
print "Optimized weights:", [K.eval(i).T for i in w]

# Restore weights from beginning:
for l in model.layers:
    l.set_weights(store_w[l])
print "\nRestored weights:", [K.eval(i).T for i in w]

print "\nSecond approach:"
sess = K.get_session()
sess.run(tf.global_variables_initializer())
for i in range(1000):
    sess.run(op, {x : x_train, y_true : y_train})
    
print "Optimized weights:", [K.eval(i).T for i in w]

Our model is y = 3 * x1 + 2 * x2 + 5
Data generated.
Initial weights: [array([[ 0.25437474,  0.50860047]], dtype=float32), array([ 0.], dtype=float32)]

First approach:
Optimized weights: [array([[ 3.0004077 ,  2.00035262]], dtype=float32), array([ 4.99958515], dtype=float32)]

Restored weights: [array([[ 0.25437474,  0.50860047]], dtype=float32), array([ 0.], dtype=float32)]

Second approach:
Optimized weights: [array([[ 3.00082374,  2.00068712]], dtype=float32), array([ 4.99917507], dtype=float32)]


### Concatenating two models and propagating the gradient.

In [5]:
%clear -f
import keras.backend as K
from keras.layers import Input, Dense, Lambda
from keras.models import Model
import numpy as np

x1 = Input(shape=(2,))
y = Lambda(lambda x: x ** 2)(x1)
grads1 = K.gradients(y, x1)
f1 = K.function(inputs=[x1], outputs=grads1)
model1 = Model(inputs=x1, outputs=y)

x_feed = np.random.randn(2, 2)
print "1. Model: y1 = x ** 2"
print "x =\n", x_feed
print "y =\n", model1.predict_on_batch(x_feed)
print "dy_dx =\n", f1([x_feed])[0]

print "\n2. Model: y2 = 3 * x"
x2 = Input(shape=(3, ))
y = Lambda(lambda x: 3 * x)(x2)
grads2 = K.gradients(y, x2)
f2 = K.function(inputs=[x2], outputs=grads2)
model2 = Model(inputs=x2, outputs=y)
print "y2 =\n", model1.predict_on_batch(x_feed)

print "\n3. Model: z = y2 o y1"
z = model2(model1.outputs)
in1 = model1.inputs[0]
f3 = K.function(inputs=[in1], outputs=[z])
print "z =\n", f3([x_feed])[0]
grads3 = K.gradients(z, in1)
f4 = K.function(inputs=[in1], outputs=grads3)
print "dz_dx =\n", f4([x_feed])[0]
print "6x =\n", 6 * x_feed

1. Model: y1 = x ** 2
x =
[[-0.0106578  -0.4100865 ]
 [ 0.53091092 -1.81399555]]
y =
[[  1.13588707e-04   1.68170929e-01]
 [  2.81866401e-01   3.29058003e+00]]
dy_dx =
[[-0.0213156  -0.82017297]
 [ 1.06182182 -3.6279912 ]]

2. Model: y2 = 3 * x
y2 =
[[  1.13588707e-04   1.68170929e-01]
 [  2.81866401e-01   3.29058003e+00]]

3. Model: z = y2 o y1
z =
[[  3.40766128e-04   5.04512787e-01]
 [  8.45599174e-01   9.87174034e+00]]
dz_dx =
[[ -0.0639468   -2.46051884]
 [  3.18546534 -10.88397408]]
6x =
[[ -0.0639468   -2.46051898]
 [  3.18546553 -10.88397332]]
